In [53]:
import pandas as pd
import json
from pandas.io.json import json_normalize

In [54]:
FILE = '../logs/test.json'

In [55]:
with open(FILE) as f:
    d = json.load(f)

In [56]:
logs = json_normalize(d)
logs['start_time'] = pd.to_datetime(logs['start_time'])
logs['end_time'] = pd.to_datetime(logs['end_time'])
del logs['losses_val']

In [57]:
logs.head()

,accuracy,accuracy_-1,accuracy_1,end_time,n_workers,running_mode,running_time,start_time,tag
0,0.795592,0.991687,0.583258,2018-05-22 15:04:55,2,synchronous,26.858709,2018-05-22 15:04:28,
1,0.687986,1.000000,0.316935,2018-05-22 15:35:06,2,synchronous,22.663874,2018-05-22 15:34:43,
2,0.734226,1.000000,0.414286,2018-05-22 15:38:23,2,synchronous,23.167867,2018-05-22 15:38:00,
3,0.796024,0.990291,0.573284,2018-05-22 15:42:08,2,synchronous,24.359314,2018-05-22 15:41:44,
4,0.884183,0.971797,0.782852,2018-05-22 15:46:49,2,synchronous,24.285353,2018-05-22 15:46:24,


In [58]:
times = [pd.DataFrame(d[i]['losses_val']) for i in range(len(d))]

In [59]:
times[0].head()

,loss_val,time
0,2270.469972,2018-05-22 15:04:28
1,2225.287023,2018-05-22 15:04:29
2,2185.046260,2018-05-22 15:04:29
3,2174.906931,2018-05-22 15:04:29
4,2166.709046,2018-05-22 15:04:30
